In [8]:
import pandas as pd
import numpy as np
import folium
from sklearn.cluster import DBSCAN

# --- 1. ЗАВАНТАЖЕННЯ ---
df = pd.read_csv('accident_clear_data.csv', sep=';')

# --- 2. ПОКРАЩЕНА ЛОГІКА ОЧИСТКИ (V3) ---
def clean_accident_cause_final(val):
    val = str(val).lower()

    # Критичні порушення
    if 'нетверезому' in val or 'сп\'яніння' in val or 'алкоголь' in val:
        return 'Алкоголь'

    # Швидкість
    elif 'швидкост' in val:
        return 'Швидкість'

    # Пішоходи (Додано: невстановлене місце, неочікуваний вихід)
    elif 'пішохід' in val or 'переход' in val or 'невстановленому' in val or 'неочікуваний' in val or 'вихід на' in val:
        return 'Пішохід'

    # Перехрестя та Світлофори (Додано: сигнали регулювання)
    elif 'перехрест' in val or 'пріоритет' in val or 'світлофор' in val or 'знак' in val or 'переваги' in val or 'регулювання' in val:
        return 'Перехрестя/Світлофор'

    # Маневрування
    elif 'маневрування' in val or 'розворот' in val or 'смуг' in val:
        return 'Маневрування'

    # Зустрічка (Додано: виїзд на зустрічну)
    elif 'обгін' in val or 'зустрічн' in val:
        return 'Обгін/Зустрічка'

    # Дистанція
    elif 'дистанц' in val or 'інтервал' in val:
        return 'Дистанція'

    # Невідомо
    elif 'невідомо' in val:
        return 'Невідомо'

    else:
        return 'Інше' # Тепер тут лишаться тільки пасажири і рідкісні випадки

df['Simple_Cause'] = df['mainAccidentCause'].apply(clean_accident_cause_final)

# Виводимо контрольну суму, щоб ти побачив різницю
print("--- Новий розподіл (Інше має зменшитись) ---")
print(df['Simple_Cause'].value_counts().head(8))

# --- 3. КЛАСТЕРИЗАЦІЯ ---
coords = df[['latitude', 'longitude']].values
coords_rad = np.radians(coords)

# Налаштування: 70 метрів, мінімум 5 ДТП
db = DBSCAN(eps=0.07/6371.0, min_samples=10, metric='haversine', algorithm='ball_tree').fit(coords_rad)
df['Cluster'] = db.labels_

# --- АГРЕГАЦІЯ (Без змін) ---
cluster_stats = df[df['Cluster'] != -1].groupby('Cluster').agg({
    'latitude': 'mean',
    'longitude': 'mean',
    'Simple_Cause': lambda x: x.mode()[0],
    'Hour': lambda x: x.mode()[0],
    'accidentDay': 'count'
}).rename(columns={'accidentDay': 'AccidentCount'})

# --- ВІЗУАЛІЗАЦІЯ З РОЗУМНИМ РОЗМІРОМ ---
m = folium.Map(location=[49.8397, 24.0297], zoom_start=12)

colors = {
    'Швидкість': 'red',
    'Алкоголь': 'black',
    'Перехрестя/Світлофор': 'orange',
    'Пішохід': 'purple',
    'Маневрування': 'blue',
    'Обгін/Зустрічка': 'darkred',
    'Дистанція': 'cadetblue',
    'Невідомо': 'lightgray',
    'Інше': 'green'
}

for cluster_id, row in cluster_stats.iterrows():
    cause = row['Simple_Cause']
    color = colors.get(cause, 'gray')
    count = row['AccidentCount']

    # --- НОВА ЛОГІКА РОЗМІРУ ---
    # 1. Base size (мінімальний розмір точки): 6 пікселів
    # 2. Log scaling: додаємо розмір не лінійно, а логарифмічно.
    #    np.log1p(count) перетворить 10 -> 2.4, 100 -> 4.6, 500 -> 6.2
    #    Множимо на 4, щоб було візуально помітно.
    calculated_radius = 4 + (np.log1p(count) * 4)

    # 3. Hard Cap (Жорстке обмеження): Максимум 25 пікселів
    radius = min(calculated_radius, 25)

    popup_html = f"""
    <div style="font-family: Arial; width: 160px;">
        <b>Зона #{cluster_id}</b><br>
        <span style="color:{color}"><b>{cause}</b></span><br>
        ДТП: {count}<br>
        Час пік: {row['Hour']}:00
    </div>
    """

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,       # Тепер радіус контрольований
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        weight=1,            # Тонша обводка кола
        popup=folium.Popup(popup_html, max_width=200),
        tooltip=f"{cause}: {count} ДТП"
    ).add_to(m)

m.save("lviv_accidents_fixed_size.html")
print("Карту оновлено! Гігантські кола прибрано.")

--- Новий розподіл (Інше має зменшитись) ---
Simple_Cause
Маневрування            1977
Пішохід                 1119
Швидкість               1097
Перехрестя/Світлофор     594
Невідомо                 452
Дистанція                258
Алкоголь                 223
Інше                     185
Name: count, dtype: int64
Карту оновлено! Гігантські кола прибрано.


In [43]:
# Подивимось, як розподілилися спрощені причини
print("--- Розподіл категорій ---")
print(df['Simple_Cause'].value_counts())

print("\n--- Які причини потрапили в 'Інше' (ТОП-10) ---")
# Фільтруємо ті, що стали "Інше", і показуємо оригінальний текст
gray_causes = df[df['Simple_Cause'] == 'Інше']['mainAccidentCause'].value_counts().head(10)
print(gray_causes)

--- Розподіл категорій ---
Simple_Cause
Порушення маневрування          2227
Інше                            1226
Перевищення швидкості           1073
Порушення проїзду переходів      697
Порушення проїзду перехресть     365
Недотримання дистанції           285
П'яне водіння                     32
Name: count, dtype: int64

--- Які причини потрапили в 'Інше' (ТОП-10) ---
mainAccidentCause
Невідомо                                           452
перехід у невстановленому місці                    203
невиконання вимог сигналів регулювання             128
неочікуваний вихід на проїзну частину              104
порушення правил перевезення пасажирів              55
пішоходи невиконання вимог сигналів регулювання     39
порушення техніки безпеки пасажиром                 39
виїзд на смугу зустрічного руху                     29
неочікуваний вихід на проїжджу частину              29
перехід пішоходів у невстановленому місці           21
Name: count, dtype: int64
